# Procesamiento de datasets
En este notebook están agrupados los procedimientos por los cuales se importaron, concatenaron y procesaron los sets de datos originalmente provistos:
- `dataset_es_dev.json`
- `dataset_es_train.json`
- `dataset_es_test.json`

Para generar sets de datos con los atributos relevantes y las reviews lemmatizadas y stemmatizadas, segpun el caso:
- `dataset_amazon_reviews_lemma.json`
- `dataset_amazon_reviews_stem.json`

De esta forma, se ahorra tiempo en la ejecución del notebook que resulve la consigna pedida.

### Importar las librerías necesarias

In [1]:
import pandas as pd
import numpy as np

# Garbage collector para optimizar recursos
import gc

In [2]:
# Previamente
## python -m spacy download es
## python -m spacy download es_core_news_sm

import spacy # https://spacy.io/usage/models
nlp = spacy.load('es_core_news_sm')

#Stop Words de es_core_news_sm
from spacy.lang.es.stop_words import STOP_WORDS
stopwords_spacy = list(STOP_WORDS)

# Quitamos de las stopwords palabras como 'no', 'bueno', 'buena' que nos pueden ser últiles para predecir
stopwords_spacy.remove('no') # En los comentarios negativos es muy repetida
stopwords_spacy.remove('bueno')
stopwords_spacy.remove('buena')
stopwords_spacy.remove('tarde')
stopwords_spacy.remove('temprano')
stopwords_spacy.remove('día')
stopwords_spacy.remove('días')
stopwords_spacy.remove('dia')
stopwords_spacy.remove('dias')
stopwords_spacy.remove('grandes')
stopwords_spacy.remove('general')

In [3]:
import nltk

#Stop Words de nltk
from nltk.corpus import stopwords
stopwords_nltk = set(stopwords.words('spanish'))

# Quitamos de las stopwords la palabra 'no', que nos puede ser últil para predecir
stopwords_nltk.remove('no')

import re
from nltk.tokenize import RegexpTokenizer

In [4]:
# Clase para personalizar las impresiones de consola
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### Importar y concatenar los datasets

In [5]:
# Importamos dataset de desarrollo
data_dev = pd.read_json("dataset_amazon/dataset_es_dev.json", lines = True)
print("- Cantidad de filas del set de " + color.YELLOW + "desarrollo" + color.END + ":", data_dev.shape[0])

# Importamos dataset de entrenamiento
data_train = pd.read_json("dataset_amazon/dataset_es_train.json", lines = True)
print("- Cantidad de filas del set de " + color.CYAN + "entrenamiento" + color.END + ":", data_train.shape[0])

# Importamos dataset de prueba
data_test = pd.read_json("dataset_amazon/dataset_es_test.json", lines = True)
print("- Cantidad de filas del set de " + color.GREEN + "prueba" + color.END + ":", data_test.shape[0])

- Cantidad de filas del set de desarrollo: 5000
- Cantidad de filas del set de entrenamiento: 200000
- Cantidad de filas del set de prueba: 5000


In [6]:
# Concatenamos todos
data = pd.concat([data_dev,data_train,data_test])
data = data.reset_index(drop=True)
data.tail()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
209995,es_0685611,product_es_0919143,reviewer_es_0461769,5,Para lo que cuesta es perfecto porque cumple s...,Perfectos para lo que sirven,es,wireless
209996,es_0638242,product_es_0636134,reviewer_es_0214562,5,"Material muy flexible y cómodo, acorde a la de...","Buen vendedor, rápido y fiable.",es,wireless
209997,es_0760108,product_es_0078583,reviewer_es_0043609,5,Se ve bien el grabado,medallas con bonito grabado,es,home
209998,es_0070806,product_es_0359350,reviewer_es_0258717,5,lo compré para mi bici pero finalmente se lo h...,timbre,es,sports
209999,es_0794544,product_es_0130307,reviewer_es_0651938,5,Pues este tipo de cascos si que son inalambric...,Auriculares sin cables de verdad,es,electronics


In [7]:
# Liberamos espacio de memoria
del(data_dev)
del(data_train)
del(data_test)
gc.collect()

17

### Filtrado de columnas
Nos quedamos con las columnas del dataset que son relevantes para predecir la valoración en general. Por eso se descartan las columnas de id de usuario y producto. La de categoría nos puede servir para relacionar palabras. La del idioma es redundante.

In [8]:
df = data[['product_category','review_title','review_body','stars']]
df.head()

,product_category,review_title,review_body,stars
0,wireless,Mala compra,"Malisimo, muy grande demasiado aparatoso y mal...",1
1,apparel,No lo he recibido,No he recibido el pedido no la devolución,1
2,sports,Que no aprieta bien en el manillar,"Tengo que buscar otro sistema, este no funcion...",1
3,home_improvement,Brocas de mantequilla,Utilicé las brocas de menor diámetro y se me d...,1
4,beauty,No me gusta,No me gusta su olor a viejo y aspecto malo,1


### Limpieza de datos

In [9]:
#Constante de signos de puntuación
import string
puntua = string.punctuation + '¡¿...'
excluded_pos = ['SCONJ','CCONJ','NUM','PUNCT','PRON','DET','ADP','AUX','X']

In [10]:
#Función para limpieza de datos con lemmatizer
def text_data_lemma(sentence):
    doc = nlp(sentence)
    
    clean_tokens = []
    for token in doc:
        if (token.pos_ not in excluded_pos and str(token) not in stopwords_spacy and len(token.text)>2): 
            temp = token.lemma_.strip()
            clean_tokens.append(temp.lower())
    
    return clean_tokens

In [11]:
from nltk.stem.snowball import SpanishStemmer # Permite stemmizar palabras en español
stemmer = SpanishStemmer()

#Función para limpieza de datos con stemmer
def text_data_stem(sentence):
    doc = nlp(sentence)
    
    clean_tokens = []
    for token in doc:
        if (token.pos_ not in excluded_pos and str(token) not in stopwords_spacy and len(token.text)>2): 
            temp = stemmer.stem(token.text).strip()
            clean_tokens.append(temp.lower())
    
    return clean_tokens

#### Lemmatización (40 min aprox)

In [12]:
# Limpiamos todas las reviews con lemmatizer
reviews_lemma = []
for i in df.index:
    rev = text_data_lemma(df.review_title.iloc[i] + ' ' + df.review_body.iloc[i])
    reviews_lemma.append(" ".join(rev))
reviews_lemma[:5]

['malo compro malisimo grande aparatoso protector pantalla',
 'recibir recibir pedido devolución',
 'apretar manillar tener buscar sistema funcionar abrazadera agarrar manillar apretar deslizar linterna abajo',
 'brocas mantequilla utilicé broca menor diámetro doblar ser mantequilla mínimo presión doblado enderezarla partir exteriormente amarillo interior fundido gris compra',
 'gustar gustar olor viejo aspecto malo']

#### Stemmización (40 min aprox)

In [13]:
# Limpiamos todas las reviews con stemmizer
reviews_stem = []
for i in df.index:
    rev = text_data_stem(df.review_title.iloc[i] + ' ' + df.review_body.iloc[i])
    reviews_stem.append(" ".join(rev))
reviews_stem[:5]

['mal compr malisim grand aparat protector pantall',
 'recib recib ped devolu',
 'apriet manill teng busc sistem funcion abrazader agarr manill apriet desliz lintern abaj',
 'broc mantequill utilic broc menor diametr dobl fues mantequill minim presion dobl enderez part exterior amarill interior fund gris compr',
 'gust gust olor viej aspect mal']

### Guardar nuevos datos en achivo json

In [14]:
# Agregamos columna al dataset
df['revs_lemma'] = reviews_lemma
df['revs_stem'] = reviews_stem
df.head()

<ipython-input-14-a1af3c2ac77a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['revs_lemma'] = reviews_lemma
<ipython-input-14-a1af3c2ac77a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['revs_stem'] = reviews_stem


,product_category,review_title,review_body,stars,revs_lemma,revs_stem
0,wireless,Mala compra,"Malisimo, muy grande demasiado aparatoso y mal...",1,malo compro malisimo grande aparatoso protecto...,mal compr malisim grand aparat protector pantall
1,apparel,No lo he recibido,No he recibido el pedido no la devolución,1,recibir recibir pedido devolución,recib recib ped devolu
2,sports,Que no aprieta bien en el manillar,"Tengo que buscar otro sistema, este no funcion...",1,apretar manillar tener buscar sistema funciona...,apriet manill teng busc sistem funcion abrazad...
3,home_improvement,Brocas de mantequilla,Utilicé las brocas de menor diámetro y se me d...,1,brocas mantequilla utilicé broca menor diámetr...,broc mantequill utilic broc menor diametr dobl...
4,beauty,No me gusta,No me gusta su olor a viejo y aspecto malo,1,gustar gustar olor viejo aspecto malo,gust gust olor viej aspect mal


In [15]:
# Guardamos dataset lemmatizado
df.to_json(path_or_buf='dataset_amazon_clean.json')